In [138]:
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pandas as pd 
import os
import re
from string import punctuation
from datetime import datetime

DIRNAME = 'data/stc510/questions'

english_stopwords = set(stopwords.words('english') + list(punctuation))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Krista\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Krista\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Krista\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


if need to turn into list: qlist = df.values.tolist() 

In [226]:
df = pd.read_json('data/stc510/questions/jeopardy.json')

In [ ]:
df

In [196]:
qeasy = df['value'] <'$2000'
qeasy

0          True
1          True
2          True
3          True
4          True
          ...  
216925    False
216926    False
216927    False
216928    False
216929    False
Name: value, Length: 216930, dtype: bool

In [211]:
df.loc[qeasy, 'show_number'] = 0


In [198]:
qeasylist = qeasy.values.tolist()

In [181]:
qhard = df['value'] >= '$2000'
qhard

0         False
1         False
2         False
3         False
4         False
          ...  
216925     True
216926     True
216927     True
216928     True
216929    False
Name: value, Length: 216930, dtype: bool

In [210]:
df.loc[qhard, 'show_number'] = 1


In [184]:
qhardlist = qhard.values.tolist()

In [185]:
corpuseasy = ' '.join([str(x) for x in qeasylist])

In [186]:
corpushard = ' '.join([str(x) for x in qhardlist])

In [187]:
easywords = [WordNetLemmatizer().lemmatize(x) for x in word_tokenize(corpuseasy)
             if x not in english_stopwords]

In [188]:
hardwords = [WordNetLemmatizer().lemmatize(x) for x in word_tokenize(corpushard)
             if x not in english_stopwords]

In [219]:
newdf = pd.DataFrame({'easy': easywords,
                      'hard': hardwords})

In [220]:
x_train, x_test, y_train, y_test = train_test_split(newdf.easy, newdf.hard, random_state=1)

In [221]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
x_train_tf = tfidf_vectorizer.fit_transform(x_train)
x_test_tf = tfidf_vectorizer.transform(x_test)

In [222]:
naive_bayes = MultinomialNB()
naive_bayes.fit(x_train_tf, y_train)
predictions = naive_bayes.predict(x_test_tf)

In [223]:
print('Accuracy: ', accuracy_score(y_test, predictions))

Accuracy:  0.9826305017240425
